### 2.6.1 SocketChannel

#### 一. 何为SocketChannel
1. SocketChannel相当于阻塞IO中的Socket, 创建SocketChannel有2种方式
    * 自己打开一个SocketChannel, 并连接到一个Server
    * 当ServerSocketChannel接受一个连接请求时, 自动创建一个SocketChannel
 
#### 二. Opening a SocketChannel
```java
SocketChannel socketChannel = SocketChannel.open();
socketChannel.connect(new InetSocketAddress("http://jenkov.com", 80));
```

#### 三. Closing a SocketChannel
```java
socketChannel.close();    
```

#### 四. Reading from a SocketChannel
1. 使用`socketChannel.read(buffer)`
2. `socketChannel.read(buffer)`方法返回读到了多少字节的数据. 如果返回-1, 表示读到了流的末尾(表示连接已关闭)
```java
ByteBuffer buf = ByteBuffer.allocate(48);
int bytesRead = socketChannel.read(buf);
```

#### 五. Writing to a SocketChannel
1. 使用`socketChannel.write(buffer)`
2. 同FileChannel一样, 不能保证一次性将buffer的所有数据写入到channel, 因此需要把该方法写在循环中,直到将buffer的数据写完. 判断条件为`buffer.hasRemaining()`

```java
String newData = "New String to write to file";

ByteBuffer buf = ByteBuffer.allocate(48);
buf.clear(); // 切换写模式
buf.put(newData.getBytes());

buf.flip();  // 切换读模式

while(buf.hasRemaining()) {
    channel.write(buf);
}
```


#### 六. 非阻塞模式的SocketChannel
1. 如果设置SocketChannel为非阻塞模式, 则connect(),read(),write()方法变为异步执行
    * `connect()`: 
      非阻塞下的connect()会在连接建立前返回, 可以使用`finishConnect()`方法判断连接是否建立成功
    ```java
    socketChannel.configureBlocking(false);
    socketChannel.connect(new InetSocketAddress("http://jenkov.com", 80));

    while(! socketChannel.finishConnect() ){
        //wait, or do something else...    
    }
    ```
    * `write()`:
      非阻塞模式下的write可能在没有写入任何数据到channel中就返回了, 因此, 需要将write()方法写在循环中, 但是先前我们已经这样做了, 所以这里不会用任何变化
    * `read()`:
      非阻塞模式下, 可能在未读到任何数据的情况下就返回, 因此要注意read方法返回的结果, 他会告诉我们读到的字节数


### 2.6.2 ServerSocketChannel

#### 一. 普通ServerSocketChannel的工作模式
1. ServerSocketChannel可以绑定到一个TCP端口等待连接
2. 等待连接:
    * 当调用`serverSocketChannel.accept()`, 就会一直阻塞等待一个客户端连接, 一旦有一个客户端连接成功就返回, 并创建一个与之对应的SocketChannel
    * 因为不想只接受一个连接, 所以把`serverSocketChannel.accept()`写在循环中
    ```java
    ServerSocketChannel serverSocketChannel = ServerSocketChannel.open();
    serverSocketChannel.socket().bind(new InetSocketAddress(9999));

    while(true){
        SocketChannel socketChannel = serverSocketChannel.accept();

        //do something with socketChannel...
    }
    ```
    
#### 二. 非阻塞模式下的ServerSocketChannel
1. 非阻塞模式下的ServerSocketChannel, 即使没有任何一个客户端连接成功, 也会立即返回. 因此, `serverSocketChannel.accept()`方法可能会返回null, 需要在循环中做出判断
    ```java
    ServerSocketChannel serverSocketChannel = ServerSocketChannel.open();

    serverSocketChannel.socket().bind(new InetSocketAddress(9999));
    serverSocketChannel.configureBlocking(false);

    while(true){
        SocketChannel socketChannel = serverSocketChannel.accept();
        if(socketChannel != null){
            //do something with socketChannel...
            }
    }

    ```